# SkySat-snow pipeline

## Define settings and paths in directory

In [ ]:
import os
import sys

# Define paths in directory
site_name = "JacksonCreek"
date = "20240420"
code_dir = '/Users/raineyaberle/Research/PhD/SnowDEMs/skysat-snow'

ref_dem_fn = f'/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/{site_name}/refdem/USGS_LPC_ID_FEMAHQ_2018_D18_merged_filtered.tif'
dem_fn = f'/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/{site_name}/{date}/{site_name}_{date}_DEM.tif'
multispec_dir = f'/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/{site_name}/{date}/{site_name}_{date}_4band_mosaic.tif'

# gcp_fn = f'/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/ITD_Functional_Class/ITD_HWY_21.shp'
# gcp_elev = 0
gcp_fn = f'/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/JacksonCreek/snotel/JacksonCreek_snotel_site_info.gpkg'
gcp_elev = 1.448

out_dir = f'/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/{site_name}/{date}'
res = 2 # spatial resolution of outputs [m]
gcp_buffer = 3 # buffer for GCP geospatial file [m]

# Check that input files and directories exist
if not os.path.exists(ref_dem_fn):
    print('Reference DEM file not found, please correct ref_dem_fn before continuing.')
if not os.path.exists(dem_fn):
    print('Input DEM file not found, please correct pc_fn before continuing.')
if not os.path.exists(gcp_fn):
    print('GCP geospatial file not found, please correct roads_fn before continuing.')
if not os.path.exists(multispec_dir):
    print('Multispectral images folder not found, please correct multispec_dir before continuing.')
out_dir = os.path.join(out_dir, 'skysat_snow')
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    print('Made directory for outputs:', out_dir)

# Make results directories
preprocess_dir = os.path.join(out_dir, 'preprocess')
masks_dir = os.path.join(out_dir, 'land_cover_masks')
corr_coreg_diff_dir = os.path.join(out_dir, 'corr_coreg_diff')
for folder in [preprocess_dir, masks_dir, corr_coreg_diff_dir]:
    if not os.path.exists(folder):
        os.mkdir(folder)

# Add path to pipeline utilities
sys.path.append(os.path.join(code_dir, 'scripts'))
import pipeline_utils as f

## Run pipeline

In [ ]:
##### 1. PREPROCESS INPUT FILES #####
print('\n1. PREPROCESS THE INPUT FILES')
# Mosaic 4-band SR imagery, preprocess roads and reference DEM
multispec_mosaic_fn, gcp_adj_fn, ref_dem_adj_fn = f.preprocess_multispec_refdem_gcp(multispec_dir, ref_dem_fn, gcp_fn, 
                                                                                    gcp_buffer, preprocess_dir)


##### 2. CONSTRUCT LAND COVER MASKS #####
print('\n2. CONSTRUCT LAND COVER MASKS')
multispec_mosaic_fn, trees_mask_fn, snow_mask_fn, gcp_mask_fn, ss_mask_fn = f.construct_land_cover_masks(multispec_mosaic_fn, 
                                                                                                         gcp_adj_fn, masks_dir, 
                                                                                                         gcp_buffer,
                                                                                                         ndvi_threshold=0.1,
                                                                                                         ndsi_threshold=0.0)


##### 3. CORRECT, COREGISTER, AND DIFFERENCE DEM #####
print('\n3. CORRECT, COREGISTER, AND DIFFERENCE DEM')
final_dem_fn, final_ddem_fn = f.correct_coregister_difference(dem_fn, ref_dem_fn, ss_mask_fn, trees_mask_fn, gcp_mask_fn, 
                                                              gcp_elev, corr_coreg_diff_dir, mask_trees=False, plot_results=True)


##### 4. PLOT FINAL DEM AND dDEM #####
print('\n4. PLOT THE FINAL DEM AND dDEM')
f.plot_dem_ddem(final_dem_fn, final_ddem_fn, gcp_mask_fn, os.path.join(out_dir, 'final_dem_ddem.png'), vmin=-5, vmax=5)


In [ ]:
# ## Estimate the appropriate length scale for calculating slope and aspect
# import skgstat as skg
# import xdem
# import numpy as np
# from scipy.optimize import minimize
# from skgstat.util.likelihood import get_likelihood

# refdem = xdem.DEM(ref_dem_fn)
# refdem.set_nodata(np.nan)
# x = np.ravel(refdem.coords()[0])
# y = np.ravel(refdem.coords()[1])
# values = np.ravel(refdem.data)
# # remove no data vaalues
# ireal = np.argwhere(~np.isnan(values))
# x, y, values = x[ireal].ravel(), y[ireal].ravel(), values[ireal].ravel()
# coords = np.concatenate([x.reshape(-1,1), y.reshape(-1,1)], axis=1)
# # select a subsample to decrease data storage needed
# n = 10000
# coords, values = coords[0:n, :], values[0:n]

# # Calculate the experimental variogram
# print('Calculating variogram')
# V = skg.Variogram(coords, values, normalize=False, n_lags=25, use_nugget=True)

# # Maximum Likehood fit
# # base initial guess on separating distance and sample variance
# sep_mean = np.nanmean(V.distance)
# sam_var = np.nanvar(V.values)
# print(f"Mean sep. distance:  {sep_mean.round(1)}    sample variance: {sam_var.round(1)}")
# # create initial guess
# #    mean dist.  variance    5% of variance
# p0 = np.array([sep_mean, sam_var, 0.1 * sam_var])
# print('initial guess: ', p0.round(1))
# # create the bounds to restrict optimization
# bounds = [[0, V.bins[-1]], [0, 3*sam_var], [0, 2.9*sam_var]]
# print('bounds:        ', bounds)
# # load the likelihood function for this variogram
# likelihood = get_likelihood(V)
# # minimize the likelihood function
# res = minimize(likelihood, p0, bounds=bounds, method='SLSQP')
# # use 100 steps
# x = np.linspace(0, V.bins[-1], 100)
# # apply the maximum likelihood fit parameters
# y_ml = V.model(x, *res.x)
# # apply the trf fit
# y_trf = V.fitted_model(x)
# # apply Levelberg marquard
# V.fit_method = 'lm'
# y_lm = V.fitted_model(x)
# # apply parameter ml
# V.fit_method = 'ml'
# y_pml = V.fitted_model(x)

# plt.plot(V.bins, V.experimental, '.b', label='experimental')
# plt.plot(x, y_trf, '-b', label='SciKit-GStat TRF')
# plt.show()

## Apply terrain correction

In [ ]:
# # Test several terrain variables
# terrain_vars = ['elevation', 'slope', 'aspect', 'maximum_curvature']
# for terrain_var in terrain_vars:
#     print(terrain_var)
#     terrain_bias = xdem.coreg.TerrainBias(terrain_var).fit(refdem, dem_after, ss_mask)
#     dem_corrected = terrain_bias.apply(dem_after)

#     ddem_before = dem_after - refdem
#     ddem_after = dem_corrected - refdem

#     ddem_before_roads = ddem_before[roads_mask]
#     print(f'NMAD before = {np.round(xdem.spatialstats.nmad(ddem_before_roads), 3)} m')

#     ddem_after_roads = ddem_after[roads_mask]
#     print(f'NMAD after = {np.round(xdem.spatialstats.nmad(ddem_after_roads), 3)} m')
#     ddem_after_roads_med = np.nanmedian(ddem_after_roads.data)
#     dem_corrected -= ddem_after_roads_med
#     ddem_after -= ddem_after_roads_med

#     fig, ax = plt.subplots(1, 2, figsize=(12,6))
#     ddem_before.plot(cmap='coolwarm_r', vmin=-5, vmax=5, ax=ax[0])
#     ax[0].set_title('dDEM')
#     ddem_after.plot(cmap='coolwarm_r', vmin=-5, vmax=5, ax=ax[1])
#     ax[1].set_title('Corrected dDEM')
#     plt.suptitle(terrain_var)
#     plt.show()

## Try correcting for directional bias

In [ ]:
# for angle in np.linspace(0, 360, 5)[0:-1]:
#     dir_bias = xdem.coreg.DirectionalBias(angle=angle).fit(refdem, dem_after, ss_mask)
#     dem_corrected = dir_bias.apply(dem_after)
#     ddem = dem_corrected - refdem
#     fig, ax = plt.subplots(1, 1, figsize=(6,6))
#     ddem.plot(cmap='coolwarm_r', vmin=-5, vmax=5, ax=ax)
#     ax.set_title(angle)
#     plt.show()